In [5]:
%reload_ext autoreload
%autoreload 2   

import sys
sys.path.append('../artifactory/')

In [6]:
import torch
import warnings

# stop warnings
torch.set_float32_matmul_precision("high")
warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [8]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#autheticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path="config.json",
)

Found the config file in: config.json


In [10]:
# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Compute target not found.")

In [ ]:
from azure.ai.ml import Environment

myenv = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="conda-yamls/pydata.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(myenv)

In [6]:
from azure.ai.ml import command, Input

# define the command
# TODO: later - add config parameters to input
command_job = command(
    code="./src_jobs",
    command="python train.py --tser_tsdb ${{inputs.tser_tsdb}}",
    environment="myenv",
    inputs={
        "tser_tsdb": Input(
            type="uri_folder",
            path="https://m3mlopssadev.blob.core.windows.net/azureml/UI/2024-01-02_161358_UTC/",
        )
    },
    compute="cpu-cluster",
)

In [ ]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path="azureml://jobs/{}/outputs/artifacts/paths/model/".format(returned_job.name),
    name="run-model-example",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL
)

ml_client.models.create_or_update(run_model)